# Download HYCOM files for the LAPS project

Author: Johanna Wren

email: johanna.wren@noaa.gov

Date: January 29, 2024

## Description

This code accesses HYCOM files through opendap, subsets the area, and saves the u and v velocity to a file as 3-hourly files

## Import libraries

In [2]:
import pandas as pd
import xarray as xr

## Read in the HYCOM dataset
Subset the 10 and 20 meter depth layers and one time slice. The time decoding throws errors so I set the `decode_times` flag to False when reading in the file. 

The `assign_coords` function here changes the longitude from -180/180 to 0-360 so we won't have a cut down the dateline. 

In [ ]:
# Read in the file through opendap
hycom = xr.open_dataset("https://tds.hycom.org/thredds/dodsC/GLBv0.08/expt_53.X/data/1994", decode_times=False)
# Select the 10 and 20 meter deph layer and only variables water_u and water_v
# I have manually set the time here, but ideally this would be done in a loop. 
hy = hycom.sel(depth=[10,20], time=-52572)[['water_u', 'water_v']]
# Change longitude from -180/180 to 0-360, then sort the data by lon to get a continuous axis
hy360 = hy.assign_coords({"lon": (hy.lon % 360)}).sortby(hy.lon)
hy360

In [ ]:
time[0:3]

Subset spatially

In [ ]:
# Set your spatial extent
min_lat=-5
max_lat=50
min_lon=120
max_lon=230

In [ ]:
# Subset the data spatially
hy360_sub = hy360.sel(lat=slice(min_lat,max_lat), lon=slice(min_lon,max_lon))
hy360_sub

In [ ]:
# Test to see if we the data looks right
#hy360_sub.water_v.sel(depth=20).plot()

## Save data
Saving as individual 3-hourly files

In [ ]:
hy360_sub.to_netcdf(path='/Users/jwren/Desktop/HYCOM_py_test.nc')


# LOOP 
This code is identical to the above code but it's a loop so you should technically be able to loop through all the time steps in the annual hycom file and save 3-hourly files

In [ ]:
# Read in the data
hycom = xr.open_dataset("https://tds.hycom.org/thredds/dodsC/GLBv0.08/expt_53.X/data/1994", decode_times=False)

# Get the time variable values
time = hycom.time.values

# Set your spatial extent
min_lat=-5
max_lat=50
min_lon=120
max_lon=230

for x in time[0:3]:
    print(x)
    
    # Select the 10 and 20 meter deph layer and only variables water_u and water_v
    hy = hycom.sel(depth=[10,20], time=x)[['water_u', 'water_v']]
    # Change longitude from -180/180 to 0-360, then sort the data by lon to get a continuous axis
    hy360 = hy.assign_coords({"lon": (hy.lon % 360)}).sortby(hy.lon)
    
    # Subset the data spatially
    hy360_sub = hy360.sel(lat=slice(min_lat,max_lat), lon=slice(min_lon,max_lon))
    
    # Save to file
    hy360_sub.to_netcdf(path=f'/Users/jwren/Desktop/HYCOM_py_test_{x}.nc')

    

-52572.0
